## Queries on FASTianF1 graph

# 1: Complete names of the 20 drivers with the most pit stops made with a single constructor

In [ ]:
PREFIX : <https://www.dei.unipd.it/db2/groupProject/FASTianF1#>
SELECT ?name (?cname AS ?constructorName) (SUM(?maxNumb) AS ?totalPitStops) WHERE{
    {
        SELECT ?partecipation ?name ?cname (MAX(?numb) AS ?maxNumb) WHERE { 
            ?driver :hasForename ?fname ;
                    :hasSurname ?sname .
            ?partecipation :hasDriver ?driver ;
                           :hasPitStop ?stop ;
                           :hasConstructor ?constructor .
            ?constructor :hasName ?cname .
            ?stop :hasStopNumber ?numb .
            BIND(CONCAT(?fname, " ", ?sname) AS ?name).
        }
        GROUP BY ?partecipation ?name ?cname
    }
}
GROUP BY ?name ?cname
ORDER BY DESC(?totalPitStops)
LIMIT 20

# 2: Most payed driver of 2023 season

In [ ]:
PREFIX : <https://www.dei.unipd.it/db2/groupProject/FASTianF1#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?name (MAX(?sal) as ?salary) WHERE {
    ?season :hasYear "2023"^^xsd:integer;
            a :Season.
    ?rating :inSeason ?season;
            :hasDriver ?driver;
            :hasSalary ?sal;
          	a :Rating.
    ?driver a :Driver;
            :hasForename ?fname;
            :hasSurname ?sname.
    BIND(CONCAT(?fname, " ", ?sname) AS ?name).
}
GROUP BY (?name)
ORDER BY DESC (?salary)

# 3: All the english (Great Britain) drivers who won an english grand prix, driving an english car, ordered by number of wins

In [ ]:
PREFIX : <https://www.dei.unipd.it/db2/groupProject/FASTianF1#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX countries: <http://eulersharp.sourceforge.net/2003/03swap/countries#>
SELECT ?name (COUNT(?name) as ?numOfWins) ?consName WHERE { 
    ?race :hasCircuit ?circuit;
          a :Race.
    ?circuit :hasLocation ?location;
             a :Circuit.
    ?location :hasCountry countries:gb;
              a :Location.
    ?racePart :partecipatedInRace ?race;
              :hasPositionOrder "1"^^xsd:integer;
              :hasConstructor ?constructor;
              :hasDriver ?driver.
    ?constructor a :Constructor;
                 :hasName ?consName;
                 :hasNation countries:gb.
    ?driver a :Driver;
            :hasNation countries:gb;
            :hasForename ?fname;
            :hasSurname ?sname.
    BIND(CONCAT(?fname, " ", ?sname) AS ?name).
} GROUP BY ?name ?consName
ORDER BY DESC (?numOfWins)

# 4: The 5 most winning drivers in F1 history, ordered by number of grand prix wins

In [ ]:
PREFIX : <https://www.dei.unipd.it/db2/groupProject/FASTianF1#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?name (COUNT(?name) as ?numOfWins) WHERE { 
    ?race :hasCircuit ?circuit;
          a :Race.
    ?racePart :partecipatedInRace ?race;
              :hasPositionOrder "1"^^xsd:integer;
              :hasDriver ?driver.
    ?driver a :Driver;
            :hasForename ?fname;
            :hasSurname ?sname.
    BIND(CONCAT(?fname, " ", ?sname) AS ?name).
} GROUP BY ?name
ORDER BY DESC (?numOfWins)
LIMIT 5

# 5: Circuits ordered by number of accidents in races in history

In [ ]:
PREFIX : <https://www.dei.unipd.it/db2/groupProject/FASTianF1#>
SELECT ?circuitName (COUNT(?circuitName) AS ?numberOfAccidents) WHERE{
    ?racePart a :RacePartecipation;
          :partecipatedInRace ?race;
          :hasStatus ?stat.
    ?stat a :Status;
          :hasName "Accident".
    ?race a :Race;
          :hasCircuit ?circ.
    ?circ a :Circuit;
          :hasName ?circuitName.
}
GROUP BY (?circuitName)
ORDER BY DESC (?numberOfAccidents)

# 6: Data about fastest qualifying laps in F1 history (both q1, q2 and q3)

In [ ]:
PREFIX : <https://www.dei.unipd.it/db2/groupProject/FASTianF1#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?driversName ?qTime ?circuitName ?raceName WHERE {
    {
        ?qualPart :hasQ3Time ?qTime.
    }
    UNION
    {
        ?qualPart :hasQ2Time ?qTime.
    }
    UNION
    {
        ?qualPart :hasQ1Time ?qTime.
    }
    ?qualPart :hasDriver ?driver;
              :partecipatedInQualif ?quali;
              a :QualifPartecipation.
    ?quali a :Qualifying.
    ?race a :Race;
          :hasA ?quali;
          :hasName ?raceName;
          :hasCircuit ?circ.
    ?circ a :Circuit;
             :hasName ?circuitName.
    ?driver a :Driver;
            :hasForename ?fname;
            :hasSurname ?sname.
    BIND(CONCAT(?fname, " ", ?sname) AS ?driversName).
    FILTER(?qTime != "00:00:00"^^xsd:time)
}
ORDER BY (?qTime)
LIMIT 100

# 7: Information regarding the youngest drivers who have won races, in order of age

In [ ]:
PREFIX : <https://www.dei.unipd.it/db2/groupProject/FASTianF1#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?fullName ?age (COUNT(?pos) AS ?wins) (GROUP_CONCAT(DISTINCT YEAR(?raceDate) ; separator=",") AS ?year) (GROUP_CONCAT(DISTINCT ?circName ; separator=",") AS ?circuitNames) WHERE{
    ?part :hasDriver ?driver ;
          :hasPosition ?pos ;
              :partecipatedInRace ?race .
    ?race :hasDate ?raceDate ;
          :hasCircuit ?circ .
    ?circ :hasName ?circName .
    ?driver :hasDateOfBirth ?dob ;
            :hasForename ?fname ;
            :hasSurname ?lname .
    BIND(CONCAT(?fname, " ", ?lname) AS ?fullName)
    BIND((YEAR(?raceDate) - YEAR(?dob)) AS ?age)
    FILTER(?pos = 1 && ?age < 26)
}
GROUP BY ?fullName ?age
ORDER BY ASC (?age)

# 8: Constructors with most wins in the constructors' championship

In [ ]:
PREFIX : <https://www.dei.unipd.it/db2/groupProject/FASTianF1#>
SELECT ?name (COUNT(?name) AS ?totalWins) (GROUP_CONCAT(DISTINCT ?outerYear ; separator=",") AS ?years) WHERE{
    {
        SELECT ?name ?race (AVG(?pos) AS ?finalPosition) ?outerYear WHERE{
            ?stand :hasRace ?race ;
                   :hasConstructor ?cons ;
                   :hasTotalPosition ?pos .
            ?race :hasRound ?outerRound ;
                  :inSeason ?outerSeason .
            ?outerSeason :hasYear ?outerYear .
            ?cons :hasName ?name .
            FILTER(?outerRound = ?maxRound && ?outerYear = ?year && ?pos = 1)
            {
                SELECT ?year (MAX(?round) AS ?maxRound) WHERE {
                    ?race :inSeason ?season ;
                          :hasRound ?round .
                    ?season :hasYear ?year .
                }
                GROUP BY ?year
            }
        }
        GROUP BY ?name ?race ?outerYear
        ORDER BY (?outerYear)
    }
}
GROUP BY ?name
ORDER BY DESC (?totalWins)

# 9: Are drivers who won the championship after 2000 older than those who won it before 2000, on average?

In [ ]:
PREFIX : <https://www.dei.unipd.it/db2/groupProject/FASTianF1#>
ASK WHERE{
    {
        SELECT (AVG(?age) as ?avgOldAge) WHERE{
            ?stand :hasRace ?race ;
                   :hasDriver ?driv ;
                   :hasTotalPosition ?pos .
            ?driv :hasDateOfBirth ?dob .
            ?race :hasDate ?raceDate ;
                  :inSeason ?outerSeason ;
                  :hasRound ?outerRound .
            ?outerSeason :hasYear ?outerYear .
            BIND((YEAR(?raceDate) - YEAR(?dob)) AS ?age)
            FILTER(?outerRound = ?maxRound && ?outerYear = ?year && ?pos = 1)
            {
                SELECT ?year (MAX(?round) AS ?maxRound) WHERE {
                    ?race :inSeason ?season ;
                          :hasRound ?round .
                    ?season :hasYear ?year .
                    FILTER(?year < 2000)
                }
                GROUP BY ?year
            }
        }
    }
    {
        SELECT (AVG(?age) as ?avgRecentAge) WHERE{
            ?stand :hasRace ?race ;
                   :hasDriver ?driv ;
                   :hasTotalPosition ?pos .
            ?driv :hasDateOfBirth ?dob .
            ?race :hasDate ?raceDate ;
                  :inSeason ?outerSeason ;
                  :hasRound ?outerRound .
            ?outerSeason :hasYear ?outerYear .
            BIND((YEAR(?raceDate) - YEAR(?dob)) AS ?age)
            FILTER(?outerRound = ?maxRound && ?outerYear = ?year && ?pos = 1)
            {
                SELECT ?year (MAX(?round) AS ?maxRound) WHERE {
                    ?race :inSeason ?season ;
                          :hasRound ?round .
                    ?season :hasYear ?year .
                    FILTER(?year >= 2000)
                }
                GROUP BY ?year
            }
        }
    }
    FILTER(?avgRecentAge > ?avgOldAge)
}

# 10: Winners of driver's championship and constructor's championship for each season.

In [ ]:
PREFIX : <https://www.dei.unipd.it/db2/groupProject/FASTianF1#>
SELECT ?outerYear ?driverName ?constructorName WHERE{
    ?driverStand :hasTotalPosition ?pos ;
                 :hasRace ?race ;
                 :hasDriver ?driver .
    ?constStand :hasTotalPosition ?pos ;
                :hasRace ?race ;
                :hasConstructor ?const .
    ?race :hasRound ?outerRound ;
          :inSeason ?season .
    ?season :hasYear ?outerYear .
    ?driver :hasForename ?fname ;
            :hasSurname ?sname .
    ?const :hasName ?constructorName .
    BIND(CONCAT(?fname, " ", ?sname) AS ?driverName).
    FILTER(?outerRound = ?maxRound && ?outerYear = ?year && ?pos = 1)
    {
        SELECT ?year (MAX(?round) AS ?maxRound) WHERE {
            ?race :inSeason ?season ;
                  :hasRound ?round .
            ?season :hasYear ?year .
        }
        GROUP BY ?year
    }
}
ORDER BY DESC (?outerYear)